# Dataset Creation - CTA/MRA/MRI

Ce notebook crée les datasets de cubes 3D pour l'entraînement du modèle.

> **📌 Note importante** : Ce notebook utilise un **échantillon de 20 séries DICOM** pour démonstration.
> 
> Le projet original sur Kaggle a traité **4000+ séries** avec la même architecture.
> Cette version permet de visualiser la structure des datasets et de tester le code localement.

**Étapes** :
1. Chargement des données localisateurs
2. Filtrage par modalité (CTA, MRA, MRI T1post, MRI T2)
3. Vérification de la disponibilité des données
4. Test sur un exemple
5. Extraction de cubes positifs (contenant anévrisme) et négatifs
6. Sauvegarde au format .npz
7. Résumé des datasets créés

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from tqdm import tqdm

# Import du package src
sys.path.append("../")

from src import (
    SERIES_DIR,
    TRAIN_CSV,
    TRAIN_LOCALIZERS_CSV,
    PROCESSED_DIR,
    print_config
)
from src.data import ajouter_Modality
from src.preprocessing import preprocessing_volume_and_coords
from src.visualization import show_middle_slices, show_slice_with_point
from src.bricks import Preprocessor, DatasetBuilder

## 1. Configuration et chargement des données

In [ ]:
# Affichage de la configuration automatique
print_config()

In [ ]:
# Chargement DataFrames (chemins automatiques)
df_loc = pd.read_csv(TRAIN_LOCALIZERS_CSV)
df_train = pd.read_csv(TRAIN_CSV)

# Ajout modalité
df_loc = ajouter_Modality(df_loc, df_train)

print(f"Total localizers: {len(df_loc)}")
print(f"\nModalités:")
print(df_loc['Modality'].value_counts())

## 2. Filtrage par modalité

Choisissez la modalité à traiter (CTA, MRA, MRI T1post, MRI T2)

In [ ]:
# Sélection modalité
MODALITY = "CTA"  # Changez selon besoin: "CTA", "MRA", "MRI T1post", "MRI T2"

df_modality = df_loc[df_loc['Modality'] == MODALITY].reset_index(drop=True)
print(f"Nombre de {MODALITY}: {len(df_modality)}")

## 3. Vérification de la disponibilité des données

Avant de traiter les données, vérifions quels patients sont réellement disponibles.

In [ ]:
# Vérifier combien de patients de la modalité sont disponibles localement
available_patients = []
for i in range(len(df_modality)):
    series_uid = df_modality.iloc[i]['SeriesInstanceUID']
    patient_path = os.path.join(SERIES_DIR, series_uid)
    if os.path.exists(patient_path):
        available_patients.append(series_uid)

print(f"Patients {MODALITY} dans le CSV: {len(df_modality)}")
print(f"Patients {MODALITY} disponibles localement: {len(available_patients)}")

if len(available_patients) > 0:
    print(f"\n✓ {len(available_patients)} patients prêts pour le traitement")
else:
    print(f"\n⚠️ Aucun patient disponible localement")
    print(f"Les données DICOM doivent être téléchargées dans: {SERIES_DIR}")

## 4. Test sur un exemple

In [ ]:
# Test sur le premier patient disponible
if len(available_patients) > 0:
    # Utiliser le premier patient disponible identifié à l'étape 3
    series_uid = available_patients[0]
    patient_path = os.path.join(SERIES_DIR, series_uid)
    
    print(f"Patient sélectionné: {series_uid}")
    
    try:
        volume, aneurysm_coords = preprocessing_volume_and_coords(
            SERIES_DIR, patient_path, df_modality
        )
        
        print(f"Volume shape: {volume.shape}")
        print(f"Aneurysm coordinates: {aneurysm_coords}")
        
        # Visualisation
        show_middle_slices(volume)
        show_slice_with_point(volume, aneurysm_coords, plane="axial")
        
    except Exception as e:
        print(f"Erreur lors du traitement: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"⚠️ Aucun patient {MODALITY} disponible localement")
    print(f"Exécutez d'abord l'étape 3 pour vérifier les données disponibles.")

## 5. Création des datasets par modalité

Cette section utilise la classe `DatasetBuilder` pour créer un dataset par modalité :
- **CTA** : Angiographie par tomodensitométrie
- **MRA** : Angiographie par résonance magnétique
- **MRI T1post** : IRM T1 avec contraste
- **MRI T2** : IRM T2

Chaque dataset contient :
- Des cubes positifs (contenant des anévrismes) 
- Des cubes négatifs (sans anévrisme)
- Des vecteurs de position (one-hot encoding)
- Les labels et patient IDs

In [ ]:
# Initialiser le preprocessor et le dataset builder
preprocessor = Preprocessor()
builder = DatasetBuilder(
    preprocessor=preprocessor,
    cube_size=48,
    series_dir=SERIES_DIR
)

print(f"DatasetBuilder initialisé: {builder}")
print(f"\nPrêt à construire les datasets pour toutes les modalités")

## 6. Sauvegarde des datasets

In [ ]:
# Sauvegarde de tous les datasets créés
# Décommenter après la création des datasets

# if 'datasets' in locals() and len(datasets) > 0:
#     print("Sauvegarde des datasets...\n")
#     
#     for modality, dataset in datasets.items():
#         # Nom de fichier standardisé
#         filename = f"{modality.lower().replace(' ', '_')}_dataset.npz"
#         output_path = os.path.join(PROCESSED_DIR, filename)
#         
#         # Sauvegarder
#         builder.save(dataset, output_path)
#         
#         # Statistiques
#         print(f"\n{modality} Dataset:")
#         print(f"  Fichier: {filename}")
#         print(f"  Total cubes: {len(dataset['cubes'])}")
#         print(f"  Positifs: {dataset['labels'].sum():.0f}")
#         print(f"  Négatifs: {(1 - dataset['labels']).sum():.0f}")
#         print(f"  Balance: {dataset['labels'].mean():.2%} positive")
#     
#     print(f"\n{'='*70}")
#     print(f"✓ {len(datasets)} datasets sauvegardés dans {PROCESSED_DIR}")
#     print(f"{'='*70}")
# else:
#     print("⚠️ Aucun dataset à sauvegarder.")
#     print("Exécutez d'abord la cellule de création des datasets.")

## 7. Résumé des datasets créés

Visualisation finale des datasets créés et leurs caractéristiques.

In [ ]:
# Résumé final des datasets créés
# Décommenter pour afficher le résumé

# if 'datasets' in locals() and len(datasets) > 0:
#     import pandas as pd
#     
#     # Créer un tableau récapitulatif
#     summary_data = []
#     for modality, dataset in datasets.items():
#         summary_data.append({
#             'Modalité': modality,
#             'Total Cubes': len(dataset['cubes']),
#             'Positifs': int(dataset['labels'].sum()),
#             'Négatifs': int((1 - dataset['labels']).sum()),
#             'Balance (%)': f"{dataset['labels'].mean()*100:.1f}%",
#             'Fichier': f"{modality.lower().replace(' ', '_')}_dataset.npz"
#         })
#     
#     df_summary = pd.DataFrame(summary_data)
#     print("\n" + "="*80)
#     print("RÉSUMÉ DES DATASETS CRÉÉS")
#     print("="*80 + "\n")
#     print(df_summary.to_string(index=False))
#     print("\n" + "="*80)
#     print(f"Total: {len(datasets)} datasets créés")
#     print(f"Localisation: {PROCESSED_DIR}")
#     print("="*80)
# else:
#     print("Aucun dataset créé pour le moment.")

## 6. Sauvegarde

In [ ]:
# Sauvegarde du dataset (décommenter après la création)
# if 'dataset' in locals():
#     output_path = os.path.join(PROCESSED_DIR, f"{MODALITY}_dataset.npz")
#     builder.save(dataset, output_path)
#     
#     # Afficher les statistiques du dataset
#     print(f"\n{'='*60}")
#     print(f"Dataset Statistics:")
#     print(f"{'='*60}")
#     print(f"Total cubes: {len(dataset['cubes'])}")
#     print(f"Cube shape: {dataset['cubes'][0].shape}")
#     print(f"Positive samples: {dataset['labels'].sum():.0f}")
#     print(f"Negative samples: {(1 - dataset['labels']).sum():.0f}")
#     print(f"Balance: {dataset['labels'].mean():.2%} positive")
# else:
#     print("⚠️ Aucun dataset à sauvegarder. Exécutez d'abord la cellule de création.")